In [1]:
import pandas as pd
import scipy as scp
import numpy as np
import time

from copy import deepcopy
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.metrics import accuracy_score,precision_score,f1_score,recall_score,classification_report,make_scorer
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.feature_selection import SelectKBest,SelectFpr
from sklearn.pipeline import Pipeline


In [2]:
df_main = pd.read_csv('Ultrasound_1.csv')
df = deepcopy(df_main)
print(df['Pass/Fail'].value_counts()/df.shape[0])
y = df.pop('Pass/Fail').values



-1    0.933418
 1    0.066582
Name: Pass/Fail, dtype: float64


In [3]:
class Processing_Df:
    def __init__(self,df_lcl):
        self.df_cl = df_lcl
        self.lst_fl = []
        self.lst_int = []
        self.lst_o = []
        self.stats_lst = []
        self.lst_d_m_c = []
        
#@@@@@@@@@@@@@@@@@@@@@@@@@$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    def df_num(self):
        #self.df_cl = self.Drop_mis_column()
        for cols in self.df_cl:
            if self.df_cl[cols].dtypes == 'float':
                self.lst_fl.append(cols)
            if self.df_cl[cols].dtypes == 'int':
                self.lst_int.append(cols)
            if self.df_cl[cols].dtypes == 'object':
                self.lst_o.append(cols)    
            
        return (self.lst_fl,self.lst_int,self.lst_o)
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    def df_std_removal(self):
        
        df_stats = self.df_cl.describe()
        for i in df_stats.columns:
            if df_stats.loc['std',i] == 0:
                self.stats_lst.append(i)
        self.df_cl.drop(columns=self.stats_lst,inplace=True) 
        return (self.df_cl)
#-------------------------------->>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
    def Drop_mis_column(self):
        dct_df = {cols:(self.df_cl[cols].isnull().sum()/self.df_cl.shape[0])*100 for cols in df.columns}
        for j in dct_df.keys():
            if dct_df[j] > 25.0:
                self.lst_d_m_c.append(j)
        self.df_cl.drop(columns=self.lst_d_m_c,inplace=True)
        return (self.df_cl)
#&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
    def Standardisation(self):
        tmp = []
        for i in  self.df_cl.columns:
            if self.df_cl[i].max()>3:
                tmp.append(i)
            else:  
                continue
        std_sc = StandardScaler()
        self.df_cl[tmp] = std_sc.fit_transform( self.df_cl[tmp])
        return  self.df_cl
#-----------------------><><><><><><><><<><>-------------------------------------------------------------
    def Mis_Treat(self):
        for i in self.df_cl.columns:
            if self.df_cl[i].isnull().sum() != 0:
                if (self.df_cl[i].skew() > 1) | (self.df_cl[i].skew() < -1):
                    self.df_cl[i].fillna(value=self.df_cl[i].median(),inplace=True)
                elif (self.df_cl[i].skew() < 1) | (self.df_cl[i].skew() > -1):
                     self.df_cl[i].fillna(value=self.df_cl[i].mean(),inplace=True)
        return self.df_cl
#--------------------------->>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<------------------------------------------
    def Replacing_outliers(self,tmp):
        df_tmp_zscore = scp.stats.zscore(self.df_cl)
        
        for i in df_tmp_zscore.columns:
            if tmp > 0:
                if (df_tmp_zscore[i]>tmp).any():
                    arr1 = np.where(df_tmp_zscore[i]>tmp)
                    
                    tmp1 = tmp*self.df_cl[i].std() + self.df_cl[i].mean()
                    for j in arr1[0]:
                        self.df_cl.loc[j,i] =  tmp1
            else:
                if (df_tmp_zscore[i]<tmp).any():
                    arr1 = np.where(df_tmp_zscore[i]<tmp)
                   
                    tmp1 = -tmp*self.df_cl[i].std() + self.df_cl[i].mean()
                    for j in arr1[0]:
                        
                        self.df_cl.loc[j,i] =  tmp1
        return self.df_cl
#-------------------------><><><><MAIN><MAIN><><><><><><><>><><><><><><><><---------------------------------------------    def         
    def main(self):
        self.Drop_mis_column()
        self.df_std_removal()
        self.df_cl = self.Mis_Treat()
        self.Replacing_outliers(3.0)
        self.Replacing_outliers(-3.0)
        self.Standardisation()
        return self.df_cl 

In [4]:
df_o = Processing_Df(df)
l_f,l_int,l_o = df_o.df_num()
df_new  =  df_o.main()
df_new.isnull().sum()
#sns.scatterplot(data = df_new,x = range(df.shape[0]),y = )

0      0
1      0
2      0
3      0
4      0
      ..
585    0
586    0
587    0
588    0
589    0
Length: 440, dtype: int64

In [5]:
x = df_new
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)
df_t = X_train.describe()

In [6]:
# lr=LogisticRegression(C = 0.1, random_state = 42, solver = 'liblinear')
dt=DecisionTreeClassifier()
rf=RandomForestClassifier()
sel_feat = SelectFpr() 
# gnb=GaussianNB()
# knn = KNeighborsClassifier(n_neighbors=3)
# svm = svm.SVC(kernel='linear')

In [ ]:
parameters_grid = [{'feat' : [sel_feat],
                    'feat__alpha' : [0.1,0.08,0.05,0.02]
                    },
                   {'classifier' : [dt],
                   'classifier__max_depth' : [5,15,3],
                   'classifier__criterion' : ['gini' , 'entropy'],
                   #'max_features' : ['log2' , 'sqrt']
                  },
                  
                  { 'classifier' : [rf],
                    'classifier__n_estimators' : [50,500,50],
                    'classifier__max_depth' : [5,15,3],
                   'classifier__criterion' : ['gini' , 'entropy'],
                   #'max_features' : ['log2' , 'sqrt']
                  }]
ppln = Pipeline([('feat',sel_feat),('classifier',dt)])

In [ ]:
model_hy_tun = GridSearchCV(ppln, param_grid=parameters_grid, cv = 5, 
                            scoring = {'accuracy': make_scorer(accuracy_score),
                                       're_call': 'recall'},refit='re_call')
model_hy_tun.fit(X_train,y_train)
print(model_hy_tun.best_score_)
print(model_hy_tun.best_params_)
print(model_hy_tun.best_estimator_)


In [10]:
import tensorflow as tf


ModuleNotFoundError: No module named 'tensorflow'